### Make Simple Prompts taken from T2I-CompBench Dataset

In [1]:
import json
import os

DATASET_FOLDER = "./prompt_dataset" 

FILES = {
    "Attribute": "color.txt",   # or 'attribute.txt' if you renamed it
    "Spatial": "spatial.txt",
    "Numeracy": "numeracy.txt"
}

LIMIT_PER_CATEGORY = 100
OUTPUT_FILE = "./simple_prompts.json"

all_prompts = []

print(f"Reading prompts from '{DATASET_FOLDER}'...")

for category, filename in FILES.items():
    file_path = os.path.join(DATASET_FOLDER, filename)
    
    if not os.path.exists(file_path):
        print(f"[ERROR] File not found: {file_path}")
        continue
        
    print(f"Processing {category} ({filename})...")
    
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            lines = [line.strip() for line in f.readlines() if line.strip()]
            
        selected = lines[:LIMIT_PER_CATEGORY]
        
        for i, prompt_text in enumerate(selected):
            all_prompts.append({
                "id": f"{category.split()[0]}_{i}",
                "category": category,
                "simple_prompt": prompt_text
            })
            
    except Exception as e:
        print(f"Error reading {filename}: {e}")

with open(OUTPUT_FILE, "w") as f:
    json.dump(all_prompts, f, indent=2)

print(f"\nSuccess! Saved {len(all_prompts)} prompts to '{OUTPUT_FILE}'.")

Reading prompts from './prompt_dataset'...
Processing Attribute (color.txt)...
Processing Spatial (spatial.txt)...
Processing Numeracy (numeracy.txt)...

Success! Saved 300 prompts to './simple_prompts.json'.


### Make Smart Prompts Using Gemini API/LLava LLM

In [ ]:
import json
import time
import os
from google import genai
from google.genai import types

client = genai.Client(api_key="")
MODEL_ID = "gemini-2.5-flash"

def get_smart_prompt(simple_prompt):
    generate_content_config = types.GenerateContentConfig(
        temperature=0.7,
        top_p=0.9,
        max_output_tokens=72,
        thinking_config=types.ThinkingConfig(thinking_budget=0),        
        system_instruction=[types.Part.from_text(
            text="You are an expert prompt engineer specializing in optimizing prompts for Text-to-Image models.\n"
                 "Rewrite the user's simple prompt into a detailed description that forces the model to generate the image accurately.\n"
                 "Output should fit within 50 tokens.\n"
                 "Return ONLY the final rewritten prompt."
        )],
    )

    try:
        response = client.models.generate_content(
            model=MODEL_ID,
            contents=f"Rewrite this prompt: '{simple_prompt}'",
            config=generate_content_config
        )
        print("response: ", response.text.strip())
        return response.text.strip()

    except Exception as e:
        print(f"  [Error] Gemini API failed: {e}")
        return simple_prompt


In [3]:
input_file = "simple_prompts.json"
output_file = "smart_prompts.json"

if not os.path.exists(input_file):
    print(f"Error: '{input_file}' not found. Run Step 1 first.")

with open(input_file, "r") as f:
    data = json.load(f)

smart_data = []
print(f"Enhancing {len(data)} prompts using {MODEL_ID}...")

for i, item in enumerate(data):
    print(f"[{i+1}/{len(data)}] Processing ID: {item['id']}...")
    
    complex_text = get_smart_prompt(item['simple_prompt'])
    time.sleep(0.5)

    smart_data.append({
        "id": item['id'],
        "category": item['category'],
        "simple_prompt": item['simple_prompt'],
        "smart_prompt": complex_text
    })

with open(output_file, "w") as f:
    json.dump(smart_data, f, indent=2)

print(f"\nDone! Smart prompts saved to '{output_file}'")

Enhancing 300 prompts using gemini-2.5-flash...
[1/300] Processing ID: Attribute_0...
[2/300] Processing ID: Attribute_1...
[3/300] Processing ID: Attribute_2...
[4/300] Processing ID: Attribute_3...
[5/300] Processing ID: Attribute_4...
[6/300] Processing ID: Attribute_5...
[7/300] Processing ID: Attribute_6...
[8/300] Processing ID: Attribute_7...
[9/300] Processing ID: Attribute_8...
[10/300] Processing ID: Attribute_9...
[11/300] Processing ID: Attribute_10...
[12/300] Processing ID: Attribute_11...
[13/300] Processing ID: Attribute_12...
[14/300] Processing ID: Attribute_13...
[15/300] Processing ID: Attribute_14...
[16/300] Processing ID: Attribute_15...
[17/300] Processing ID: Attribute_16...
[18/300] Processing ID: Attribute_17...
[19/300] Processing ID: Attribute_18...
[20/300] Processing ID: Attribute_19...
[21/300] Processing ID: Attribute_20...
[22/300] Processing ID: Attribute_21...
[23/300] Processing ID: Attribute_22...
[24/300] Processing ID: Attribute_23...
[25/300] Pr